In [ ]:
#| default_exp pinataapi

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Pinata
> Python API for interacting with Pinata

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
import requests
import json

In [ ]:
#| ignoretest
with open("creds.json") as f:
    
    creds = json.loads(f.read())["Pinata"]["JWT"]

In [ ]:
#| export
def generate_apikey(cred:str,#JWT
                    key_name:str, #Key name
                    pinlist:bool=False,#list pins
                    userPinnedDataTotal:bool=False, #total data stored
                    hashMetadata:bool=True, #metadata
                    hashPinPolicy:bool=False, #policy
                    pinByHash:bool=True, #pin cid
                    pinFileToIPFS:bool=True,#upload file to IPFS
                    pinJSONToIPFS:bool=True,#upload json to IPFS
                    pinJobs:bool=True,#see pin jobs
                    unpin:bool=True,#unpin ipfs cid
                    userPinPolicy:bool=True #establish pin policy
                   
):

    url = "https://api.pinata.cloud/users/generateApiKey"

    payload = json.dumps({
      "keyName": key_name,
      "permissions": {
        "endpoints": {
          "data": {
            "pinList": pinlist,
            "userPinnedDataTotal": userPinnedDataTotal
          },
          "pinning": {
            "hashMetadata": hashMetadata,
            "hashPinPolicy": hashPinPolicy,
            "pinByHash": pinByHash,
            "pinFileToIPFS": pinFileToIPFS,
            "pinJSONToIPFS": pinJSONToIPFS,
            "pinJobs": pinJobs,
            "unpin": unpin,
            "userPinPolicy": userPinPolicy
          }
        }
      }
    })
    headers = {
      'Authorization': f'Bearer {cred}',
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response


Generate API Key

In [ ]:
#| ignoretest
response = generate_apikey(creds,"Test")
response.json()

{'pinata_api_key': '443e2dadf1e6a47c2754',
 'pinata_api_secret': '0f84d67fe64136d4d9ddbfb1235a8ab1fd490f12c1f6fcc532b758062695a1dc',
 'JWT': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VySW5mb3JtYXRpb24iOnsiaWQiOiJkMjhmMjBjZi1kZmUwLTRjOGYtOTI3Mi0yNmI5YzJkOGEzY2QiLCJlbWFpbCI6InZpbnRhZ2Vnb2xkMTIzQGdtYWlsLmNvbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJwaW5fcG9saWN5Ijp7InJlZ2lvbnMiOlt7ImlkIjoiTllDMSIsImRlc2lyZWRSZXBsaWNhdGlvbkNvdW50IjoxfV0sInZlcnNpb24iOjF9LCJtZmFfZW5hYmxlZCI6ZmFsc2UsInN0YXR1cyI6IkFDVElWRSJ9LCJhdXRoZW50aWNhdGlvblR5cGUiOiJzY29wZWRLZXkiLCJzY29wZWRLZXlLZXkiOiI0NDNlMmRhZGYxZTZhNDdjMjc1NCIsInNjb3BlZEtleVNlY3JldCI6IjBmODRkNjdmZTY0MTM2ZDRkOWRkYmZiMTIzNWE4YWIxZmQ0OTBmMTJjMWY2ZmNjNTMyYjc1ODA2MjY5NWExZGMiLCJpYXQiOjE2NTg0OTk3NDF9.Rd4F1zBE8wTPIwi4mD1x3pdEnfEM2Ik7zjiw0vQ7AC8'}

In [ ]:
#| export
def list_apikeys(cred:str
):

    url = "https://api.pinata.cloud/users/apiKeys"

    payload={}
    headers = {
      'Authorization': f'Bearer {cred}'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    
    return response


List API Keys

In [ ]:
#| ignoretest
newest_apikey = list_apikeys(creds).json()["keys"][0];newest_apikey

{'id': '1ff9d58d-d7fa-4598-9863-c53a806faff6',
 'name': 'Test',
 'key': '443e2dadf1e6a47c2754',
 'secret': 'ed7a3bdc89fb353b6c4f4155a9adb7f5:d15adb31f816663e2612ff5f89a5497abc30a5100ab83575247ba67cc9d49a7d61293d77c94049a8d83a82d8f02423357f311b26204bb57e7be9181d324e6424',
 'max_uses': None,
 'uses': 0,
 'user_id': 'd28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd',
 'scopes': {'endpoints': {'data': {'pinList': False,
    'userPinnedDataTotal': False},
   'pinning': {'unpin': True,
    'pinJobs': True,
    'pinByHash': True,
    'hashMetadata': True,
    'hashPinPolicy': False,
    'pinFileToIPFS': True,
    'pinJSONToIPFS': True,
    'userPinPolicy': True}}},
 'revoked': False,
 'createdAt': '2022-07-22T14:22:21.191Z',
 'updatedAt': '2022-07-22T14:22:21.191Z'}

In [ ]:
#| export
def revoke_apikey(cred:str,
                  revoke_apikey:str
):
    url = "https://api.pinata.cloud/users/revokeApiKey"

    payload = json.dumps({
      "apiKey": revoke_apikey
    })
    headers = {
      'Authorization': f'Bearer {cred}',
      'Content-Type': 'application/json'
    }

    response = requests.request("PUT", url, headers=headers, data=payload)

    return response


Revoke API Key

In [ ]:
#| ignoretest
revoke_apikey(creds,response.json()["JWT"])

<Response [200]>

In [ ]:
#| export
def upload_file(cred:str,#JWT key
                name:str, #filename
                fpaths:list, #filepaths
                metadata:dict, #metadata
                cid_version:str="1", #IPFS cid
                directory:bool=False #upload directory
):

    pinataMetadata = dict({"name":name,"keyvalues":{}})
    pinataMetadata["name"] = name
    pinataMetadata["keyvalues"].update(metadata)
    
    pinataOptions = dict({"cidVersion":cid_version,"directory":directory})
    

    url = "https://api.pinata.cloud/pinning/pinFileToIPFS"

    payload={"pinataOptions":json.dumps(pinataOptions),"pinataMetadata":json.dumps(pinataMetadata)}
    
    if directory:
        print("feature is not ready yet")
    
    files=[('file',(name,open(fpaths,'rb'),'application/octet-stream'))]
    
    headers = {
      'Authorization': f'Bearer {cred}'
    }

    response = requests.request("POST", url, headers=headers, data=payload, files=files)

    return response

Upload File to IPFS with metadata

In [ ]:
#| ignoretest
metadata = {"type":"AdultData"}

upload_file(creds,"adult_data.csv","output/adult_data.csv",metadata,cid_version=1,directory=False).text

'{"IpfsHash":"bafybeibcsnxt2sqzsvfg4lzg3iuxkfit37l3rz5gqmdxwsgec56qalmxie","PinSize":3975284,"Timestamp":"2022-06-17T22:43:08.303Z","isDuplicate":true}'

In [ ]:
#| export
def upload_jsonfile(cred:str,#JWT key
                name:str, #filename
                fpaths:list, #filepaths
                metadata:dict, #metadata
                cid_version:str, #IPFS cid
                directory:bool=False #upload directory
):

    url = "https://api.pinata.cloud/pinning/pinJSONToIPFS"

    payload = json.dumps({
      "pinataOptions": {
        "cidVersion": cid_version
      },
      "pinataMetadata": {
        "name": name,
        "keyvalues": metadata
      },
      "pinataContent": {"file":fpaths}
    })
    headers = {
        'Authorization': f'Bearer {cred}',
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response

This endpoint is optimized for JSON files

In [ ]:
#| ignoretest
metadata = {"name":"Vote"}

upload_jsonfile(creds,"ens_airdrop_Nov8th2021.json","output/ens_airdrop_Nov8th2021.json",metadata,cid_version=1,directory=False).text

'{"IpfsHash":"bafkreid56by3qspgyxvppq3ggj2x4tec3akpnlffbhangjzzfc3lfh66yq","PinSize":45,"Timestamp":"2022-07-22T14:29:51.674Z","isDuplicate":true}'

In [ ]:
#| export
def pin(cred:str,#JWT key
        cid:str, #IPFS cid
        fn=None, #Name of file
        pinataMetadata=None #Add keys and values associated with IPFS CID
):

    url = "https://api.pinata.cloud/pinning/pinByHash"

    payload = json.dumps({
      "hashToPin": cid,
      "pinataMetadata": {
        "name": fn,
        "keyvalues": pinataMetadata
      }
    })
    headers = {
      'Authorization': f'Bearer {cred}',
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    print(response.text)


Pin files

In [ ]:
#| ignoretest
metadata = {"dApp":"Ethereum Name Service",
            "token":"ENS"
           }

pin(creds,"QmUUSHH2ycqciruPaRcptpUQDDeRiMV3G9PxA6KUupXTJV",fn="ens_airdrop",pinataMetadata=metadata)

{"id":"8cba5496-3cd4-4edb-92d7-023c0ce9c145","ipfsHash":"QmUUSHH2ycqciruPaRcptpUQDDeRiMV3G9PxA6KUupXTJV","status":"prechecking","name":"ens_airdrop"}


In [ ]:
#| export
def unpin(cred:str,#JWT Key
          cid:str #IPFS CID
):
    
    url = f"https://api.pinata.cloud/pinning/unpin/{cid}"

    payload={}
    headers = {
      'Authorization': f'Bearer {cred}'
    }

    response = requests.request("DELETE", url, headers=headers, data=payload)

    print(response)

Unpin IPFS CID

In [ ]:
#| ignoretest
unpin(creds,"QmUUSHH2ycqciruPaRcptpUQDDeRiMV3G9PxA6KUupXTJV")

<Response [200]>


If IPFS CID isn't pinned. A 500 error message will be returned

In [ ]:
#| ignoretest
unpin(creds,"QmZnxARhJWsCbTxiAzoRhnxHgMtoEkNJNS8DGLCBEMvm4V")

<Response [500]>


In [ ]:
#| export
def edit_metadata(cred:str,#JWT Key
              cid:str, #IPFS CID
              name:str, #filename
              metadata=None #Add keys and values associated with IPFS CID
):

    url = "https://api.pinata.cloud/pinning/hashMetadata"
    
    pinataMetadata = dict({"name":name,"keyvalues":{}})
    pinataMetadata["keyvalues"].update(metadata)
    pinataMetadata["ipfsPinHash"] = cid

    payload = json.dumps(pinataMetadata)
    headers = {
      'Authorization': f'Bearer {cred}',
      'Content-Type': 'application/json'
    }

    response = requests.request("PUT", url, headers=headers, data=payload)

    return response


Edit metadata of already pinned IPFS CID

In [ ]:
#| ignoretest
metadata = {"type":"Test"}

edit_metadata(creds,"bafybeibcsnxt2sqzsvfg4lzg3iuxkfit37l3rz5gqmdxwsgec56qalmxie","name",metadata)

<Response [200]>

If IPFS CID is not actively pinned. A 500 error message will be returned

In [ ]:
#| ignoretest
metadata = {"type":"Test"}

edit_metadata(creds,"QmUUSHH2ycqciruPaRcptpUQDDeRiMV3G9PxA6KUupXTJV","name",metadata)

<Response [500]>

In [ ]:
#| export

def get_pinned_jobs(cred:str,# JWT Key
                    params=None # filtering pinned jobs
):

    '''
    'sort' - Sort the results by the date added to the pinning queue (see value options below)
    'ASC' - Sort by ascending dates
    'DESC' - Sort by descending dates
    'status' - Filter by the status of the job in the pinning queue (see potential statuses below)
    'prechecking' - Pinata is running preliminary validations on your pin request.
    'searching' - Pinata is actively searching for your content on the IPFS network. This may take some time if your content is isolated.
    'retrieving' - Pinata has located your content and is now in the process of retrieving it.
    'expired' - Pinata wasn't able to find your content after a day of searching the IPFS network. Please make sure your content is hosted on the IPFS network before trying to pin again.
    'over_free_limit' - Pinning this object would put you over the free tier limit. Please add a credit card to continue pinning content.
    'over_max_size' - This object is too large of an item to pin. If you're seeing this, please contact us for a more custom solution.
    'invalid_object' - The object you're attempting to pin isn't readable by IPFS nodes. Please contact us if you receive this, as we'd like to better understand what you're attempting to pin.
    'bad_host_node' - You provided a host node that was either invalid or unreachable. Please make sure all provided host nodes are online and reachable.
    'ipfs_pin_hash' - Retrieve the record for a specific IPFS hash
    'limit' - Limit the amount of results returned per page of results (default is 5, and max is 1000)
    'offset' - Provide the record offset for records being returned. This is how you retrieve records on additional pages (default is 0)
    '''

    base_url = 'https://api.pinata.cloud/pinning/pinJobs/'

    header = {'Authorization': f'Bearer {cred}'}

    response = requests.get(base_url, headers=header,params=params)

    return response

Retrieve pinned jobs. If there are pending pin jobs, they will be returned in json format.

In [ ]:
#| ignoretest
get_pinned_jobs(creds).json()

{'count': 0, 'rows': []}

In [ ]:
#| export
def get_pinned_files(cred:str,# JWT Key
                     params=None # Filter returned pinned files 
):

    '''
    Query Parameters = params

    hashContains: (string) - Filter on alphanumeric characters inside of pin hashes. Hashes which do not include the characters passed in will not be returned.
    pinStart: (must be in ISO_8601 format) - Exclude pin records that were pinned before the passed in 'pinStart' datetime.
    pinEnd: (must be in ISO_8601 format) - Exclude pin records that were pinned after the passed in 'pinEnd' datetime.
    unpinStart: (must be in ISO_8601 format) - Exclude pin records that were unpinned before the passed in 'unpinStart' datetime.
    unpinEnd: (must be in ISO_8601 format) - Exclude pin records that were unpinned after the passed in 'unpinEnd' datetime.
    pinSizeMin: (integer) - The minimum byte size that pin record you're looking for can have
    pinSizeMax: (integer) - The maximum byte size that pin record you're looking for can have
    status: (string) -
        * Pass in 'all' for both pinned and unpinned records
        * Pass in 'pinned' for just pinned records (hashes that are currently pinned)
        * Pass in 'unpinned' for just unpinned records (previous hashes that are no longer being pinned on pinata)
    pageLimit: (integer) - This sets the amount of records that will be returned per API response. (Max 1000)
    pageOffset: (integer) - This tells the API how far to offset the record responses. For example,
    if there's 30 records that match your query, and you passed in a pageLimit of 10, providing a pageOffset of 10 would return records 11-20.
    '''

    base_url = 'https://api.pinata.cloud/data/pinList?'

    header = {'Authorization': f'Bearer {cred}'}

    response = requests.get(base_url, headers=header,params=params)

    return response

Retrieve pinned files and use filtering arguments such as `hashContains` and `status` to filter by IPFS CID and get only the file that is pinned. Pinata keeps a log of all the times an IPFS CID is pinned and unpinned. Therefore, without the `status` filter multiple records would be returned.

In [ ]:
#| ignoretest
get_pinned_files(creds,params={"hashContains":"bafybeibcsnxt2sqzsvfg4lzg3iuxkfit37l3rz5gqmdxwsgec56qalmxie","status":"pinned"}).json()

{'count': 1,
 'rows': [{'id': 'a1a93991-99bc-4c12-9553-2e689046cf65',
   'ipfs_pin_hash': 'bafybeibcsnxt2sqzsvfg4lzg3iuxkfit37l3rz5gqmdxwsgec56qalmxie',
   'size': 3975284,
   'user_id': 'd28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd',
   'date_pinned': '2022-06-17T22:43:08.303Z',
   'date_unpinned': None,
   'metadata': {'name': 'name',
    'keyvalues': {'Type': 'Test',
     'type': 'Test',
     'company': 'Pinata',
     'filetype': 'json'}},
   'regions': [{'regionId': 'NYC1',
     'currentReplicationCount': 1,
     'desiredReplicationCount': 1}]}]}

In [ ]:
#| export
def get_datausage(cred:str,# JWT Key
                  params=None # Filter returned data usage statistics 
):

    header = {'Authorization': f'Bearer {cred}'}

    base_url = 'https://api.pinata.cloud/data/userPinnedDataTotal'

    response = requests.get(base_url, headers=header,params=params)

    return response

Retrieve data usage stats. The stats are in bytes.

In [ ]:
#| ignoretest
get_datausage(creds).json()

{'pin_count': -223,
 'pin_size_total': 349265187,
 'pin_size_with_replications_total': 349265187}

In [ ]:
#|hide
from nbdev.doclinks import *
nbdev_export()

Converted 00_utils.ipynb.
Converted 01_ipfshttpapi.ipynb.
Converted 02_estuaryapi.ipynb.
Converted 03_pinataapi.ipynb.
Converted 04_tutorial.fastai.ipynb.
Converted index.ipynb.
